# PluriGaussian simulations

<!-- SUMMARY: Categorical Simulations using PluriGaussian model. Automatic model fitting. Connectivity test (using acceptation-rejection technique)  -->

<!-- CATEGORY: Courses -->

In [ ]:
import gstlearn as gl
import gstlearn.plot as gp
import gstlearn.document as gdoc
from matplotlib.colors import ListedColormap
import numpy as np
import os

gdoc.setNoScroll()

Defining some essential parameters:

In [ ]:
ndim = 2
gl.defineDefaultSpace(gl.ESpaceType.RN, ndim);

nbsimu = 20
nbcc   = 4
cmap   = ListedColormap(['red', 'blue', 'yellow'])
figsize = (8,6)

Downloading the data base (from the distribution **Data.NF**) 

In [ ]:
path_nf = gdoc.loadData("PluriGaussian", "Data.NF")
data = gl.Db.createFromNF(path_nf)

Creating the output Grid, the Model (Cubic) and the Neighborhood (Unique):

In [ ]:
grid = gl.DbGrid.create(nx=[110,110])

model = gl.Model.createFromParam(type=gl.ECov.CUBIC, ranges=[50,30])

neigh = gl.NeighUnique()

##### Displaying Data:

Samples are represented with a different color per **facies**:
- first facies in *red*
- second facies in *blue*
- third facies in *yellow*

Samples which must belong to the same connected component (see variable **connect**) are displayed with large symbols.

In [ ]:
data

In [ ]:
fig,ax = gp.init(figsize = figsize, flagEqual=True)
ax.symbol(data, nameColor="facies", nameSize="connect", 
               edgecolors='black', sizmin=40, cmap=cmap)
ax.decoration(title="Conditioning Information")

Creating Proportions and Lithotype Rule

In [ ]:
props = gl.dbStatisticsFacies(data)
rule = gl.Rule.createFromNames(["S","S","F1","F2","F3"])

In [ ]:
gp.plot(rule, proportions = props, cmap=cmap)
gp.decoration(title="Lithotype Rule")

Calculate the Experimental Variogram of the Underlying Gaussian Random Function and fit the Model (used in PGS).

In [ ]:
varioparam = gl.VarioParam.createOmniDirection(dlag=5, nlag=20)
ruleprop = gl.RuleProp.createFromRule(rule, props)
vario = gl.variogram_pgs(data, varioparam, ruleprop)

model_gaus = gl.Model()
err = model_gaus.fit(vario, types=[gl.ECov.CUBIC], constraints=gl.Constraints(1.))

In [ ]:
res = gp.varmod(vario, model_gaus, asCov=True)

PluriGaussian Simulation

In [ ]:
err = gl.simpgs(data, grid, ruleprop, model_gaus, None, neigh, nbsimu=nbsimu,
                namconv = gl.NamingConvention("SimuPGS"))

Show several simulation outcomes

In [ ]:
fig, axs = gp.init(2,3,figsize=(20,10), flagEqual=True)
axs[0,0].raster(grid,"SimuPGS.1", cmap=cmap)
axs[0,0].symbol(data,nameColor="facies", nameSize="connect", edgecolors='black', 
                  sizmin=20, cmap=cmap)
axs[0,1].raster(grid,"SimuPGS.2", cmap=cmap)
axs[0,1].symbol(data,nameColor="facies", nameSize="connect", edgecolors='black', 
                  sizmin=20, cmap=cmap)
axs[0,2].raster(grid,"SimuPGS.4", cmap=cmap)
axs[0,2].symbol(data,nameColor="facies", nameSize="connect", edgecolors='black', 
                  sizmin=20, cmap=cmap)
axs[1,0].raster(grid,"SimuPGS.6", cmap=cmap)
axs[1,0].symbol(data,nameColor="facies", nameSize="connect", edgecolors='black', 
                  sizmin=20, cmap=cmap)
axs[1,1].raster(grid,"SimuPGS.10", cmap=cmap)
axs[1,1].symbol(data,nameColor="facies", nameSize="connect", edgecolors='black', 
                  sizmin=20, cmap=cmap)
axs[1,2].raster(grid,"SimuPGS.11", cmap=cmap)
axs[1,2].symbol(data,nameColor="facies", nameSize="connect", edgecolors='black', 
                  sizmin=20, cmap=cmap)
fig.subplots_adjust(right=0.7)

Define an Acceptation-Rejection function

Acceptation internal function: For a given simulation outcome

- Select the **Target Facies** and build its Connected Components,
- Read the indices of the connected component(s) at constraining wells,
- Return the score: **True** if these indices are similar and **False** otherwise.

In [ ]:
def accept(data, grid, name, verbose=False, transBinary=True, faccc=2):
    
    # Get the indices of samples which should be connected (starting from 0)
    rankData = [i for i in range(data.getNSample()) if data[i,"connect"] > 0]
    rankGrid = grid.locateDataInGrid(data, rankData)
    if verbose:
        print("Number of conditioning data =",len(rankData))
        print("Their ranks in the input Data Base =",rankData)
        print("Their ranks in the output Data Base =",rankGrid)
    
    # Perform the labelling into connected components
    grid.setLocator(name, gl.ELoc.Z, cleanSameLocator=True)
    err = gl.dbMorpho(grid, gl.EMorpho.CC, vmin=faccc-0.5, vmax=faccc+0.5)
    cc_list = grid[rankGrid,"Morpho*"]
    if verbose:
        print("List of their connected components indices =",cc_list)

    # Check that the data points belong to the same connected component
    number = len(np.unique(cc_list))
    return_val = (number == 1)
    if verbose:
        print("Acceptation score =",return_val)
        
    # Convert the valid Simulation outcome into a binary image
    if return_val and transBinary:
        grid[name] = grid["Morpho*"] == cc_list[0]
    
    grid.deleteColumn("Morpho*")
    return return_val

Check the acceptation / rejection function on the Simulation outcome #1.

In [ ]:
isValid = accept(data, grid, "SimuPGS.1", verbose=True, transBinary=False)

Check the acceptation function on all simulation outcomes

In [ ]:
for i in range(nbsimu):
    name = "SimuPGS." + str(i+1)
    isValid = accept(data, grid, name, False)
    if not isValid:
        grid.deleteColumn(name)
    else:
        print("Simulation ",name,"is valid")

Derive the Probability Map

In [ ]:
grid.statisticsBySample(["SimuPGS*"],[gl.EStatOption.MEAN])

In [ ]:
fig,ax = gp.init(figsize = figsize, flagEqual=True)
ax.raster(grid, "Stats.MEAN")
ax.symbol(data, nameColor="facies", nameSize="connect", edgecolors='black', sizmin=20, 
               cmap=cmap)
ax.decoration(title="Probability of Connecting Wells")